# LLM setup notebook

In [52]:
# Importing all the necessary libraries
import pandas as pd
import zipfile
import os
import json
import ollama
import re
from collections import Counter
import matplotlib.pyplot as plt

### Code to connect all the files in within the ZIP file into one DF

In [53]:
import os
import pandas as pd
import json

# Folder containing the JSON files
folder_path = "jobs"

# List to store data
data_list = []

# Iterate over each JSON file in the extracted folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.json'):  # Check if it's a JSON file
        file_path = os.path.join(folder_path, file_name)
        try:
            # Read the JSON file
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)

                df_temp = pd.DataFrame(data)
                data_list.append(df_temp)
        except Exception as e:
            print(f"Error reading {file_name}: {e}")

# Combine all DataFrames into one named df
df = pd.concat(data_list, ignore_index=True)


In [54]:
df

,Title,URL,Area,Published,Description,Job Link,HTML_Text
0,IT-tekniker til museets udstillingsteknik og d...,https://www.jobindex.dk/vis-job/h579207,Unknown,2016-01-01,I forbindelse med åbningen af flere nye udstil...,https://www.jobindexarkiv.dk/cgi/showarchive.c...,Lignende jobannoncer Job i Region Sjælland kat...
1,Senior Front End Developer,https://www.jobindex.dk/vis-job/r6559770,København,2016-01-01,Relevant details not found in the PaidJob-inne...,https://www.jobindexarkiv.dk/cgi/showarchive.c...,Lignende jobannoncer Job i Storkøbenhavn kateg...
2,Tysktalende IT-supporter/System konsulent,https://www.jobindex.dk/vis-job/r6559726,Søndersø,2016-01-01,Relevant details not found in the PaidJob-inne...,https://www.jobindexarkiv.dk/cgi/showarchive.c...,Lignende jobannoncer Job i Fyn kategoriseret i...
3,Er du en af de absolut dygtigste Test managers?,https://www.jobindex.dk/vis-job/r6559752,København,2016-01-01,Relevant details not found in the PaidJob-inne...,https://www.jobindexarkiv.dk/cgi/showarchive.c...,Lignende jobannoncer Job i Storkøbenhavn kateg...
4,Er du en af de absolut dygtigste tekniske test...,https://www.jobindex.dk/vis-job/r6559751,København,2016-01-01,Relevant details not found in the PaidJob-inne...,https://www.jobindexarkiv.dk/cgi/showarchive.c...,Lignende jobannoncer Job i Storkøbenhavn kateg...
...,...,...,...,...,...,...,...
1642,IT-servicetekniker til TELE-POST,https://www.jobindex.dk/vis-job/r6611189,Grønland,2016-01-30,Relevant details not found in the PaidJob-inne...,https://www.jobindexarkiv.dk/cgi/showarchive.c...,Lignende jobannoncer Job hos Tusass (3) Jobind...
1643,Frontend Developer,https://www.jobindex.dk/vis-job/r6610302,København,2016-01-30,Relevant details not found in the PaidJob-inne...,https://www.jobindexarkiv.dk/cgi/showarchive.c...,Lignende jobannoncer Job i Storkøbenhavn kateg...
1644,Dynamics AX Senior Konsulenter,https://www.jobindex.dk/vis-job/r6611334,København S,2016-01-31,Relevant details not found in the PaidJob-inne...,https://www.jobindexarkiv.dk/cgi/showarchive.c...,Lignende jobannoncer Job i Storkøbenhavn kateg...
1645,Programmør og systemudvikler,https://www.jobindex.dk/vis-job/r6611338,Hedehusene,2016-01-31,Relevant details not found in the PaidJob-inne...,https://www.jobindexarkiv.dk/cgi/showarchive.c...,Lignende jobannoncer Job i Storkøbenhavn kateg...


## Data Preprocessing Before LLM

In this step, we preprocess the data to ensure it is ready for the LLM. We remove rows with missing or insufficient information, as well as those that do not align with the project's objectives, ensuring that only relevant and complete data is passed for further processing.

In [55]:
# Drop rows where 'HTML_Text' is None
df = df.dropna(subset=['HTML_Text'])

# Calculate word counts for each row in the 'HTML_Text' column
df['word_count'] = df['HTML_Text'].apply(lambda x: len(x.split()))

# Generate statistics on the word counts
stats = df['word_count'].describe()

# Print the statistics
print(stats)


count    1647.000000
mean      630.154220
std       341.703805
min        66.000000
25%       417.500000
50%       670.000000
75%       826.500000
max      4080.000000
Name: word_count, dtype: float64


### We remove the HTML texts where the word count is less than 400 as those do not give use enought info to work with.

In [56]:
# Define a threshold for minimum word count
min_word_count = 400

# Filter the original dataframe in place, only keeping rows meeting the word count requirement
df = df[df['HTML_Text'].apply(lambda x: len(x.split()) > min_word_count)].copy()

df.shape

(1247, 8)

### Removing additional letters from city names. For example ''Aalborg C'' would become ''Aalborg''

In [57]:
# Function to city area indications
def clean_trailing_indicators(area):
    if pd.isna(area):
        return None
    # Remove things like 'o', 'c', 'SØ', 'N', 'K'
    area = re.sub(r'\s+[a-zA-ZÆØÅæøå]+$', '', area.strip())
    return area

# Apply the cleaning function to the 'Area' column
df['Area'] = df['Area'].apply(clean_trailing_indicators)

### Some ads have more than one city listed as an option. We only keep the city that was listed as the first option

In [58]:
# Simplified function to keep the first city
def keep_first_place_simple(area):
    if pd.isna(area):
        return None
    # Split by simpler delimiters
    delimiters = [',', '/', ' og', ' eller', ' or', ' and']
    for delim in delimiters:
        if delim in area:
            # Keep only the first part
            area = area.split(delim)[0].strip()
            break
    # Return cleaned area if it is meaningful
    return area if len(area) > 2 else None

# Apply the simplified cleaning function
df['Area'] = df['Area'].apply(keep_first_place_simple)

### Mapping the 3 cities that have different name in English vs Danish

In [59]:
# Mapping of Danish to English city names
city_name_mapping = {
    'København': 'Copenhagen',
    'Århus': 'Aarhus',
    'Helsingør': 'Elsinore'
}

# Replace Danish names with their English equivalents
df['Area'] = df['Area'].replace(city_name_mapping)

### A list of cities we want to look at. If the area is not one of the ones listed, it is removed. These are the 30 biggest cities in Denmark

In [60]:
# List of 30 biggest cities in Denmark
cities = [
    "Copenhagen", "Aarhus", "Odense", "Aalborg", "Esbjerg", "Randers", "Kolding", "Horsens", "Vejle", "Roskilde",
    "Herning", "Hørsholm", "Silkeborg", "Næstved", "Fredericia", "Viborg", "Køge", "Holstebro", "Taastrup", "Slagelse",
    "Hillerød", "Sønderborg", "Svendborg", "Hjørring", "Holbæk", "Frederikshavn", "Nørresundby", "Ringsted", "Haderslev",
    "Skive", "Ølstykke-Stenløse", "Nykøbing Falster", "Greve Strand", "Kalundborg", "Ballerup", "Rødovre", "Lyngby",
    "Albertslund", "Hvidovre", "Glostrup", "Ishøj", "Birkerød", "Farum", "Frederikssund", "Brøndby Strand",
    "Skanderborg", "Hedensted", "Frederiksværk", "Lillerød", "Solrød Strand"
]

In [61]:
# Filter the DataFrame to only include rows where Area is in the cities list
df = df[df['Area'].isin(cities)]

# Display the filtered DataFrame
print("Filtered DataFrame:")
print(df.shape)

# This is a sample data. Only for a few months

Filtered DataFrame:
(692, 8)


In [47]:
# # Install Ollama
# !sudo apt-get install -y pciutils
# !curl -fsSL https://ollama.com/install.sh | sh

In [12]:
# Sets up environment variables and starts the Ollama server
import os
import threading
import subprocess



In [62]:
SYSTEM_PROMPT = """

Extract relevant skills from a job listing, separating them into **primary IT-related technical skills** and **secondary (soft/general) skills**.

### Primary Skills
Focus on IT-specific technical skills such as:
- Programming languages, frameworks, tools, platforms, and technologies.
- Examples: Python, React, Docker, AWS, SQL, Kubernetes.

### Secondary Skills
Focus on non-technical competencies, including:
- Communication, leadership, problem-solving, teamwork, time management.

---

### Rules:
- Categorize skills into **primary** (IT-related) and **secondary** (soft skills).
- Split compound terms into separate skills (e.g., "Python and Java" → ["Python", "Java"]).
- Exclude vague, repetitive, or non-actionable terms.
- Translate all extracted skills into **English** if the input text is in Danish.
- Ensure outputs are clear, specific, and concise.


---

### Output Format:
```json
{
  "skills": {
    "primary": ["Skill 1", "Skill 2", "..."],
    "secondary": ["Skill A", "Skill B", "..."]
  }
}

---

Example:

Job: "Looking for Python, Docker, AWS, and strong communication skills."

Output:
{
    "skills": {
    "primary": ["Python", "Docker", "AWS"],
    "secondary": ["communication"]
  }
}

"""

In [ ]:
import pandas as pd
import json

# Function to extract skills (primary and secondary) from job listings
def extract_skills(html_text, row_id):
    prompt = f"""
    Extract the relevant skills from the following job listing, categorizing them into **primary (IT-related technical skills)** and **secondary (soft/general skills)**.

    ### Skills Categories:
    1. **Primary Skills:** IT-related technical skills (e.g., Python, AWS, Kubernetes, Docker, SQL, React, etc.).
    2. **Secondary Skills:** Soft skills and traits (e.g., Communication, Teamwork, Problem-Solving, Leadership).

    ### Rules:
    - Split combined skills into distinct terms (e.g., "Python and Java" → ["Python", "Java"]).
    - Focus on actionable, specific, and clearly stated skills.
    - Outputs must be in **clear and concise English only**, regardless of the input language.
    - If no skills are present, return empty lists for both categories.
    - Translate all extracted skills into **English** if the input text is in Danish.


    Provide the results in this JSON format:
    {{
      "skills": {{
        "primary": ["Skill 1", "Skill 2", "..."],
        "secondary": ["Skill A", "Skill B", "..."]
      }}
    }}

    Job Listing Text:
    {html_text}
    """
    
    url = 'http://130.225.39.216:11434/api/chat'
    headers = {'Content-Type': 'application/json'}
    data = {
        'model': 'llama3.1:latest',
        'messages': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': prompt}
        ],
        'format': 'json',
        'options': {"temperature": 0.1, "num_ctx": 8000}
    }
    
    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()  # Raise exception for response errors
        return response.json()['message']['content']
    except (json.JSONDecodeError, KeyError, requests.exceptions.RequestException) as e:
        print(f"Error for Row {row_id}: {e}")
        return {"skills": {"primary": [], "secondary": []}}

# Example usage
# row_id = example_data_row_id
# html_text = example_html_text
# result = extract_skills(html_text, row_id)
# print(result)

# Function to refine extracted skills
import requests
import json

def refine_skills(extracted_skills, row_id):
    refinement_prompt = f"""
    Refine the following extracted skills, ensuring accuracy and clear categorization:

    Extracted Skills:
    {json.dumps(extracted_skills, indent=2)}

    ### Refined Categories:
    1. **Primary Skills:** IT-related technical skills only.
    2. **Secondary Skills:** Clearly defined soft skills.

    ### Rules:
    - Keep skills actionable, specific, and IT-relevant for "primary."
    - Use concise and clear English.
    - Outputs must be in **clear and concise English.**
    - Translate all extracted skills into **English** if the input text is in Danish.

    Provide the results in this JSON format:
    {{
      "skills": {{
        "primary": ["Skill 1", "Skill 2", "..."],
        "secondary": ["Skill A", "Skill B", "..."]
      }}
    }}
    """
    
    url = 'http://130.225.39.216:11434/api/chat'
    headers = {'Content-Type': 'application/json'}
    data = {
        'model': 'llama3.1:latest',
        'messages': [
            {'role': 'system', 'content': 'SYSTEM_PROMPT'},  # Ensure SYSTEM_PROMPT is defined
            {'role': 'user', 'content': refinement_prompt}
        ],
        'format': 'json',
        'options': {"temperature": 0.1, "num_ctx": 8000},
    }

    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()  # Raise an error for a failed request
        return response.json()['message']['content']
    except (json.JSONDecodeError, KeyError, requests.exceptions.RequestException) as e:
        print(f"Error refining skills for Row {row_id}: {e}")
        return extracted_skills



In [65]:
import os

# Initialize the DataFrame with the columns for Primary_Skills and Secondary_Skills
df['Primary_Skills'] = None
df['Secondary_Skills'] = None

# Output file to store the results
output_file = 'updated_job_listings_with_skills.csv'
log_file = '[STEP5]LLM_for_project_progress_log.log'

# Create the CSV file with headers before the loop if it doesn't exist
if not os.path.exists(output_file):
    df[:0].to_csv(output_file, index=False, encoding='utf-8')  # Write only the headers initially

# Check the log file to find the last processed row
if os.path.exists(log_file):
    with open(log_file, 'r') as log:
        last_processed = int(log.read().strip())
else:
    last_processed = -1  # Start from the beginning if no log file exists

# Process job listings and add extracted skills to the DataFrame
for i, html_text in enumerate(df['HTML_Text']):
    # Skip rows that have already been processed
    if i <= last_processed:
        continue

    try:
        # Extract and refine skills
        extracted = extract_skills(html_text, row_id=i + 1)
        refined = refine_skills(extracted, row_id=i + 1)

        # Safely retrieve primary and secondary skills
        primary_skills = ", ".join(refined.get('skills', {}).get('primary', [])) or ""
        secondary_skills = ", ".join(refined.get('skills', {}).get('secondary', [])) or ""

        print(f"Primary Skills for Row {i + 1}: {primary_skills}")
        print(f"Secondary Skills for Row {i + 1}: {secondary_skills}")

        # Manually construct the row to save
        row_to_save = df.iloc[i].to_dict()  # Get the current row as a dictionary
        row_to_save['Primary_Skills'] = primary_skills
        row_to_save['Secondary_Skills'] = secondary_skills

        # Append the row to the CSV file
        pd.DataFrame([row_to_save]).to_csv(output_file, mode='a', index=False, header=False, encoding='utf-8')

        print(f"Row {i + 1} saved to {output_file}.")

        # Update the log file with the last processed row
        with open(log_file, 'w') as log:
            log.write(str(i))

    except Exception as e:
        print(f"Error processing Row {i + 1}: {e}")

        # Save the row with empty skills to the CSV
        row_to_save = df.iloc[i].to_dict()
        row_to_save['Primary_Skills'] = ""
        row_to_save['Secondary_Skills'] = ""
        pd.DataFrame([row_to_save]).to_csv(output_file, mode='a', index=False, header=False, encoding='utf-8')

        # Update the log file with the last processed row, even if it failed
        with open(log_file, 'w') as log:
            log.write(str(i))
